## 네이버 뉴스 크롤링

In [6]:
# -*- coding: utf-8 -*-
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from pprint import pprint
from tqdm.notebook import tqdm

'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
< naver 뉴스 전문 가져오기 >_select 사용
- 네이버 뉴스만 가져와서 결과값 조금 작음 
- 결과 메모장 저장 -> 엑셀로 저장 
'''''''''''''''''''''''''''''''''''''''''''''''''''''''''
RESULT_PATH = './'
now = datetime.now() #파일이름 현 시간으로 저장하기

def get_news(n_url):
    news_detail = []

    breq = requests.get(n_url, headers={'User-Agent':'Mozilla/5.0'})
    bsoup = BeautifulSoup(breq.content)

    title = bsoup.find('h2', class_='media_end_head_headline').text
    news_detail.append(title)

    pdate = bsoup.find('span', class_="media_end_head_info_datestamp_time _ARTICLE_DATE_TIME").text[:11]
    news_detail.append(pdate)

    _text = bsoup.find('div', class_='newsct_article _article_body').get_text().replace('\n', " ")
    _text = _text.replace("    동영상 뉴스   ", "")
    btext = _text.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    news_detail.append(btext.strip())
  
    news_detail.append(n_url)
    
    pcompany = bsoup.find('div', class_='copyright').find('p', class_='c_text').text
    pcompany = pcompany.split('All rights reserved.')[0].split('Copyright ⓒ')[1].strip()
    news_detail.append(pcompany)

    return news_detail

def crawler(maxpage,query,s_date,e_date):

    crawled_data = {}
    years = []
    company = []
    title = []
    contents = []
    link = []
    s_from = s_date.replace(".","")
    e_to = e_date.replace(".","")
    page = 1
    maxpage_t =(int(maxpage)-1)*10+1   # 11= 2페이지 21=3페이지 31=4페이지  ...81=9페이지 , 91=10페이지, 101=11페이지

    f = open(f"./contents_text_{query}.txt", 'w', encoding='utf-8-sig')
    
    for page in tqdm(range(1, maxpage_t+1, 10)):
    
        url = "https://search.naver.com/search.naver?where=news&query=" + query + "&sort=0&ds=" + s_date + "&de=" + e_date + "&nso=so%3Ar%2Cp%3Afrom" + s_from + "to" + e_to + "%2Ca%3A&start=" + str(page)
        
        req = requests.get(url,
                          headers={'User-Agent':'Mozilla/5.0'})
        print(url)
        cont = req.content
        soup = BeautifulSoup(cont)
        
        for urls in soup.find_all("a", class_="info"):
            try :
                if urls.get_text() == "네이버뉴스":
                    print(urls["href"])
                    news_detail = get_news(urls["href"])
                        # pdate, pcompany, title, btext
                    f.write("{}\t{}\t{}\t{}\t{}\n".format(news_detail[1], news_detail[4], news_detail[0], news_detail[2],news_detail[3]))  # new style
                    years.append(news_detail[1])
                    company.append(news_detail[4])
                    title.append(news_detail[0])
                    contents.append(news_detail[2])
                    link.append(news_detail[3])
                    
            except Exception as e:
                print(e)
                continue
        
    crawled_data["years"] = years
    crawled_data["company"] = company
    crawled_data["title"] = title
    crawled_data["contents"] = contents
    crawled_data["link"] = link
    f.close()
    return crawled_data
    
def excel_make(crawled_data):
    data = pd.DataFrame(crawled_data)
    
    xlsx_outputFileName = f'{query}_({s_date}~{e_date}).xlsx'
    data.to_excel(RESULT_PATH+xlsx_outputFileName, encoding='utf-8-sig', index=False)


maxpage = input("최대 출력할 페이지수 입력하시오: ")
if not maxpage.isdigit():
    maxpage = input("최대 출력할 페이지수 입력하시오: ")
query = input("검색어 입력: ")
s_date = input("시작날짜 입력(ex) 2021.01.01):")
e_date = input("끝날짜 입력(ex) 2021.05.24):")
crawled_data = crawler(maxpage,query,s_date,e_date) #검색된 네이버뉴스의 기사내용을 크롤링합니다. 
excel_make(crawled_data) #엑셀로 만들기

최대 출력할 페이지수 입력하시오: 1000
검색어 입력: 애플페이
시작날짜 입력(ex) 2021.01.01):2022.01.01
끝날짜 입력(ex) 2021.05.24):2022.12.06


  0%|          | 0/1000 [00:00<?, ?it/s]

https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=1
https://n.news.naver.com/mnews/article/243/0000035671?sid=105
https://n.news.naver.com/mnews/article/009/0005055954?sid=101
https://n.news.naver.com/mnews/article/001/0013624816?sid=101
https://n.news.naver.com/mnews/article/003/0011576878?sid=101
https://n.news.naver.com/mnews/article/366/0000859665?sid=101
https://n.news.naver.com/mnews/article/417/0000875388?sid=101
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=11
https://n.news.naver.com/mnews/article/001/0013611850?sid=101
https://n.news.naver.com/mnews/article/008/0004824141?sid=105
https://n.news.naver.com/mnews/article/016/0002074322?sid=101
https://n.news.naver.com/mnews/article/015/0004781982?sid=105
https://n.news.naver.com/mnews/article/243/0000035284?sid=101
https://n.news.na

https://n.news.naver.com/mnews/article/029/0002769533?sid=105
https://n.news.naver.com/mnews/article/088/0000786573?sid=105
https://n.news.naver.com/mnews/article/629/0000186841?sid=101
https://n.news.naver.com/mnews/article/277/0005184007?sid=105
https://n.news.naver.com/mnews/article/020/0003464579?sid=101
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=191
https://n.news.naver.com/mnews/article/629/0000186841?sid=101
https://n.news.naver.com/mnews/article/277/0005184007?sid=105
https://n.news.naver.com/mnews/article/020/0003464579?sid=101
https://n.news.naver.com/mnews/article/025/0003242129?sid=101
https://n.news.naver.com/mnews/article/031/0000713012?sid=101
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=201
https://n.news.naver.com/mnews/article/025/0003242129?sid=101
https://n.news

https://n.news.naver.com/mnews/article/629/0000177279?sid=105
https://n.news.naver.com/mnews/article/243/0000031705?sid=101
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=411
https://n.news.naver.com/mnews/article/243/0000031705?sid=101
https://n.news.naver.com/mnews/article/015/0004759938?sid=101
https://n.news.naver.com/mnews/article/629/0000177153?sid=101
https://n.news.naver.com/mnews/article/421/0006379098?sid=105
https://n.news.naver.com/mnews/article/092/0002270010?sid=105
https://n.news.naver.com/mnews/article/030/0003049229?sid=101
https://n.news.naver.com/mnews/article/138/0002134109?sid=105
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=421
https://n.news.naver.com/mnews/article/119/0002646060?sid=101
https://n.news.naver.com/mnews/article/092/0002270010?sid=105
https://n.news

...

https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9361
https://n.news.naver.com/mnews/article/092/0002256843?sid=105
https://n.news.naver.com/mnews/article/001/0013206505?sid=105
https://n.news.naver.com/mnews/article/003/0011170769?sid=101
https://n.news.naver.com/mnews/article/015/0004703712?sid=105
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9371
https://n.news.naver.com/mnews/article/092/0002256843?sid=105
https://n.news.naver.com/mnews/article/001/0013206505?sid=105
https://n.news.naver.com/mnews/article/003/0011170769?sid=101
https://n.news.naver.com/mnews/article/015/0004703712?sid=105
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9381
https://n.news.naver.com/mnews/article/092/00

https://n.news.naver.com/mnews/article/001/0013206505?sid=105
https://n.news.naver.com/mnews/article/003/0011170769?sid=101
https://n.news.naver.com/mnews/article/015/0004703712?sid=105
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9571
https://n.news.naver.com/mnews/article/138/0002124755?sid=105
https://n.news.naver.com/mnews/article/092/0002256843?sid=105
https://n.news.naver.com/mnews/article/001/0013206505?sid=105
https://n.news.naver.com/mnews/article/003/0011170769?sid=101
https://n.news.naver.com/mnews/article/015/0004703712?sid=105
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9581
https://n.news.naver.com/mnews/article/138/0002124755?sid=105
https://n.news.naver.com/mnews/article/092/0002256843?sid=105
https://n.news.naver.com/mnews/article/001/0013206505?sid=105
https://n.ne

https://n.news.naver.com/mnews/article/092/0002256843?sid=105
https://n.news.naver.com/mnews/article/001/0013206505?sid=105
https://n.news.naver.com/mnews/article/003/0011170769?sid=101
https://n.news.naver.com/mnews/article/015/0004703712?sid=105
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9751
https://n.news.naver.com/mnews/article/138/0002124755?sid=105
https://n.news.naver.com/mnews/article/092/0002256843?sid=105
https://n.news.naver.com/mnews/article/001/0013206505?sid=105
https://n.news.naver.com/mnews/article/003/0011170769?sid=101
https://n.news.naver.com/mnews/article/015/0004703712?sid=105
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9761
https://n.news.naver.com/mnews/article/138/0002124755?sid=105
https://n.news.naver.com/mnews/article/092/0002256843?sid=105
https://n.ne

https://n.news.naver.com/mnews/article/092/0002256843?sid=105
https://n.news.naver.com/mnews/article/001/0013206505?sid=105
https://n.news.naver.com/mnews/article/003/0011170769?sid=101
https://n.news.naver.com/mnews/article/015/0004703712?sid=105
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9931
https://n.news.naver.com/mnews/article/138/0002124755?sid=105
https://n.news.naver.com/mnews/article/092/0002256843?sid=105
https://n.news.naver.com/mnews/article/001/0013206505?sid=105
https://n.news.naver.com/mnews/article/003/0011170769?sid=101
https://n.news.naver.com/mnews/article/015/0004703712?sid=105
https://search.naver.com/search.naver?where=news&query=애플페이&sort=0&ds=2022.01.01&de=2022.12.06&nso=so%3Ar%2Cp%3Afrom20220101to20221206%2Ca%3A&start=9941
https://n.news.naver.com/mnews/article/138/0002124755?sid=105
https://n.news.naver.com/mnews/article/092/0002256843?sid=105
https://n.ne

ModuleNotFoundError: No module named 'openpyxl'

In [11]:
import pandas as pd

data = pd.read_excel(f'./{query}_({s_date}~{e_date}).xlsx')
data